In [ ]:
#| default_exp growthree

In [ ]:
#| hide
!pip install -Uq diffusers transformers fastcore
!pip install accelerate

In [ ]:
import torch,numpy as np,math,matplotlib.pyplot as plt
import torchvision.transforms as tfms
from diffusers import StableDiffusionPipeline,AutoencoderKL,UNet2DConditionModel,LMSDiscreteScheduler
from transformers import CLIPTokenizer,CLIPTextModel
from PIL import Image
import gc,os
from tqdm.auto import tqdm
from dataclasses import dataclass
from functools import partial

In [ ]:
#| hide
%matplotlib inline

In [ ]:
#| export
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def release_cache():
    gc.collect()
    torch.cuda.empty_cache()
#把图片排成 rows,cols的网格中，先排cols,后排rows
#其中len(imgs)=cols x rows
def image_grid(imgs, rows, cols):
    w,h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, img in enumerate(imgs): grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
def isColab():
    try:
        import google.colab
        IN_COLAB = True
    except:
        IN_COLAB = False
    if IN_COLAB:
        print("当前环境是Colab")
    else:
        print("当前环境不是Colab")
    return IN_COLAB
isColab(),torch_device

当前环境不是Colab


(False, 'cpu')

## 超参设定

In [ ]:
#| export
@dataclass
class HiperParameter():
    prompts:tuple
    fps:int=1
    prompt_per_second:int=1
    
    seed=1000
    guide_factor:int=7.5
    num_inference:int=5
    start_step:int=0
    
    output_dir='grow_output'
tree_prompts =(
    "An oak tree with bare branches in the winter snowing blizzard bleak",
    "A barren oak tree with no leaves and grass on the ground",
    "An oak tree in the spring with bright green leaves",
    "An oak tree in the summer with dark green leaves with a squirrel on the trunk",
    "An oak tree in the fall with colorful leaves on the ground",
    "An barren oak tree with no leaves in the fall leaves on the ground long shadows",
    "An oak tree with bare branches in the winter snowing blizzard bleak"
)
hparam=HiperParameter(prompts=tree_prompts)


"An oak tree with bare branches in the winter snowing blizzard bleak"：一棵冬天光秃秃的橡树，在暴风雪中，景色凄凉。

"A barren oak tree with no leaves and grass on the ground"：一棵光秃秃的橡树，没有叶子，地上长满了草。

"An oak tree in the spring with bright green leaves"：一棵春天长满鲜绿叶子的橡树。

"An oak tree in the summer with dark green leaves with a squirrel on the trunk"：一棵夏天叶子深绿的橡树，树干上有一只松鼠。

"An oak tree in the fall with colorful leaves on the ground"：一棵秋天落叶满地的橡树，树叶色彩斑斓。

"An barren oak tree with no leaves in the fall leaves on the ground long shadows"：一棵秋天光秃秃的橡树，地上落满了树叶，形成了长长的阴影。

"An oak tree with bare branches in the winter snowing blizzard bleak"：一棵冬天光秃秃的橡树，在暴风雪中，景色凄凉。

## 模型定义

In [ ]:
modelName="CompVis/stable-diffusion-v1-4"
mirror='bfsu' if isColab() else None

if isColab():
    pipe = StableDiffusionPipeline.from_pretrained(modelName, 
                                                   revision="fp16", 
                                                   mirror=mirror,
                                                   torch_dtype=torch.float16)
    pipe.to(torch_device)

    vae=pipe.vae
    tokenizer=pipe.tokenizer
    text_encoder=pipe.text_encoder
    unet=pipe.unet
    schedular=pipe.scheduler
else:
    vae=AutoencoderKL.from_pretrained(modelName,subfolder='vae',mirror=mirror).to(torch_device)
    tokenizer=CLIPTokenizer.from_pretrained(modelName,subfolder="tokenizer",mirror=mirror)
    text_encoder=CLIPTextModel.from_pretrained(modelName,subfolder="text_encoder",mirror=mirror).to(torch_device)
    unet=UNet2DConditionModel.from_pretrained(modelName,subfolder='unet',mirror=mirror).to(torch_device)
    schedular=LMSDiscreteScheduler(beta_schedule='scaled_linear',beta_start=0.00085, beta_end=0.012 )

当前环境不是Colab
当前环境不是Colab


In [ ]:
#| export
@torch.no_grad()
def pil2Latents(input_image:Image)-> torch.FloatTensor:
    '''
    把图片转换成vae的输入，生成的tensor已经移torch_device上了
    返回：size=[1,4,64,64]
    '''
    def tfms2Latent(r):
        return 2*r-1
    ts=tfms.ToTensor()(input_image).unsqueeze(0)
    ts=tfms2Latent(ts).to(vae.device,dtype=vae.dtype)
    return 0.18215*vae.encode(ts).latent_dist.sample()
@torch.no_grad()
def latents2Pil(latents:torch.FloatTensor) ->Image:
    '''
    把隐变量还原成PIL.Image
    latents: FloatTensor,size=[1,4,64,64]
    '''
    def tfms2Img(r):
        r=(0.5*r+0.5)
        return r.float().clamp(0,1)
    decode_img=vae.decode(latents/0.18215).sample.detach().cpu()
    decode_img=decode_img.permute(0,2,3,1).squeeze()
    decode_img=tfms2Img(decode_img)

    arr_img=decode_img.numpy()*255
    arr_img=arr_img.astype('uint8')
    return Image.fromarray(arr_img)

@torch.no_grad()
def batchPil2Latents(imgs:list)-> torch.FloatTensor:
    '''
    批量把图片转换成vae的输入，生成的tensor已经移torch_device上了
    返回：size=[B,4,64,64]
    '''
    def tfms2Latent(r):
        return 2*r-1
    f=tfms.ToTensor()
    
    ts=torch.stack([f(img) for img in imgs])
    ts=tfms2Latent(ts).to(vae.device,dtype=vae.dtype)
    return 0.18215*vae.encode(ts).latent_dist.sample()
@torch.no_grad()
def batchLatents2Pil(latents:torch.FloatTensor) ->Image:
    '''
    批量把隐变量还原成PIL.Image
    latents: FloatTensor,size=[B,4,64,64]
    '''
    def tfms2Img(r):
        r=(0.5*r+0.5)
        return r.float().clamp(0,1)
    decode_imgs=vae.decode(latents/0.18215).sample.detach().cpu()
    decode_imgs=decode_imgs.permute(0,2,3,1)
    
    decode_imgs=tfms2Img(decode_imgs)

    arr_img=decode_imgs.numpy()*255
    arr_img=arr_img.astype('uint8')
    return [Image.fromarray(a) for a in arr_img]

## 准备嵌入层

In [ ]:
!wget -O oak_tree.jpg "https://github.com/johnrobinsn/diffusion_experiments/blob/scenario1/images/oak_tree.jpg?raw=true"

In [ ]:
tree_path='oak_tree.jpg'

img=Image.open(tree_path)
img_latent=pil2Latents(img)

no_guide_ids=tokenizer("",padding='max_length',max_length=tokenizer.model_max_length,return_tensors='pt').input_ids
guide_ids=tokenizer(hparam.prompts,padding='max_length',max_length=tokenizer.model_max_length,return_tensors='pt').input_ids


with torch.no_grad():
    ids=torch.cat([no_guide_ids,guide_ids]).to(torch_device)
    token_emb=text_encoder(ids).last_hidden_state
uncondition_token_emb=token_emb[0]
prompts_token_emb=token_emb[1:]


def mix_embbed_and_uncond(i,j,wi,wj,V,u):
    i=i%len(V)
    j=j%len(V)

    v=V[i]*wi+V[j]*wj
    return torch.stack([u,v])
mix_frame_emb=partial(mix_embbed_and_uncond,V=prompts_token_emb,u=uncondition_token_emb)


**注意** 使用partial的函数，需要赋值的参数要在参数列表的最后面

## 生成每一帧图像的函数

In [ ]:
def generate_baseon_prompt_emb(img_latent:torch.FloatTensor,token_emb:torch.FloatTensor,pbar=None)->(Image,torch.FloatTensor):
    '''
        classes free model
        :param token_emb: img_latent 图像的隐变量,(2,4,64,64)的tensor
        :param token_emb: 句子经过clip_encoder后的向量表示,(2,T,D) 
        :return:  denoise 后的图片
    '''
    if hparam.seed>=0:
        torch.manual_seed(hparam.seed)
    else:
        torch.seed()

    start_step=hparam.start_step
    schedular.set_timesteps(hparam.num_inference)
    
    init_noise=torch.randn_like(img_latent)
    img_latent=schedular.add_noise(img_latent,init_noise,schedular.timesteps[start_step:start_step+1])

    if pbar is None:
        pbar=tqdm(range(len(schedular.timesteps)))
    with torch.no_grad():
        for s in range(len(schedular.timesteps)):
            if s>=start_step:
                t=schedular.timesteps[s]
                inp=torch.concat([img_latent,img_latent])
                inp=schedular.scale_model_input(inp,t)

                noise=unet(inp,t,encoder_hidden_states=token_emb).sample

                pred_noise=noise[0]+hparam.guide_factor*(noise[1]-noise[0])
                # pred_noise=pred_noise/pred_noise.norm()*noise[0].norm()
                img_latent=schedular.step(pred_noise,t,img_latent).prev_sample
            pbar.update(1)
    return latents2Pil(img_latent),img_latent


## 重点先测试这2个函数是否好用

### 观察1
固定noise,没有任何interp，树的形态没有发生大的变化。

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=17390125398225616219

gens=[]
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,1,0)
  frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
  gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

![tree](imgs/tree1.png)

### 观察2
固定noise,interp比例为0.8,0.2.树的形态没变，可以看到过度的很平缓自然

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=17390125398225616219

gens=[]
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,0.8,0.2)
  frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
  gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

![tree](imgs/tree2.png)

### 观察3
固定noise,interp比例为0.5,0.5，结论同上

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=17390125398225616219

gens=[]
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,0.5,0.5)
  frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
  gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

![tree](imgs/tree3.png)

### 观察4
不固定noise,interp比例为0.5,0.5，树的形态发生改变，看来不是我想要的

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=-1

gens=[]
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,0.5,0.5)
  frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
  gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

![tree](imgs/tree4.png)

### 观察5
固定noise,interp比例为0.5,0.5，start_step设置20，变化太少

In [ ]:
hparam.num_inference=50
hparam.start_step=20
hparam.seed=17390125398225616219

gens=[]
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,0.5,0.5)
  frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
  gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

![tree](imgs/tree5.png)

### 观察5.1
固定noise,interp比例为0.5,0.5，start_step设置0

In [ ]:
hparam.num_inference=50
hparam.start_step=0
hparam.seed=17390125398225616219

gens=[]
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,0.5,0.5)
  frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
  gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

### 异常6
使用上一帧作为输入，观察退化现象

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=17390125398225616219

gens=[]
prev_latent=img_latent.clone()
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,1,0)
  frame,prev_latent=generate_baseon_prompt_emb(prev_latent,tx)
  gens.append(frame)
  # hparam.start_step=+0
  # if i==2:break
image_grid(gens, 1, len(tree_prompts)).resize((7*256,256))

![tree](imgs/tree6.png)

### 观察 7 (改进实验)
使用上一帧作为输入，先生成一个初始化latent(prev_latent)，之后的训练start_step从0开始

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=17390125398225616219

gens=[]
prev_latent=img_latent.clone()
for i in range(len(tree_prompts)):

  tx=mix_frame_emb(i,i+1,1,0)
  frame,prev_latent=generate_baseon_prompt_emb(prev_latent,tx)
  gens.append(frame)
  hparam.start_step=+0
image_grid(gens, 1, len(tree_prompts)).resize((7*256,256))

### 观察 8 
使用初始化种子latent，两个提示词直接的过度情况。

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.seed=17390125398225616219

gens=[]

_,prev_latent=generate_baseon_prompt_emb(img_latent,mix_frame_emb(0,1,1,0))
hparam.start_step=0

rows,cols=2,8
for r in range(1,rows+1):
    w=1
    for c in range(cols):

        tx=mix_frame_emb(r,r+1,w,1-w)
        w-=1/cols
        frame,prev_latent=generate_baseon_prompt_emb(prev_latent,tx)
        gens.append(frame)

  
image_grid(gens, rows, cols).resize((cols*256,rows*256))

### 观察 9
如果全部start_step 0开始训练，每次都生成一样的图片吗？

answer:当然不会，因为text_emb每次输入的是不同的。我预计应该是与原图的风格变化比较大

In [ ]:
hparam.num_inference=50
hparam.start_step=0
hparam.seed=17390125398225616219

gens=[]
for i in range(len(tree_prompts)):

    tx=mix_frame_emb(i,i+1,0,1)
    frame,frame_latent=generate_baseon_prompt_emb(img_latent,tx)
    gens.append(frame)
image_grid(gens, 1, len(tree_prompts))

生成每一帧的代码

## 生成video

In [ ]:
! rm -rf {hparam.output_dir}

### 方法1，固定初始latent

生成30s的视频，每个提示词5s

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.fps=10
hparam.prompt_per_second=5
hparam.seed=17390125398225616219


os.makedirs(hparam.output_dir,exist_ok=True)

num_of_prompt=len(hparam.prompts)
num_of_generate_frame=(num_of_prompt-1)*hparam.prompt_per_second*hparam.fps
pbar=tqdm(range(num_of_generate_frame*hparam.num_inference))



def mapfunc(t):
    '''
        把每一帧映射到对应的prompt索引
    '''
    frac_index=t*(num_of_prompt-1)/(num_of_generate_frame-1)
    index1,index2=math.floor(frac_index),math.floor(frac_index)+1
    weight1,weight2=index2-frac_index,frac_index-index1
    
    if index2>num_of_prompt-1:
        index2=0
    return index1,index2,weight1,weight2    

#  (0, 1, 0.2941176470588235, 0.7058823529411765)
for k,e in enumerate(map(mapfunc,range(num_of_generate_frame))):
    pbar.set_description(f'Frame {k:05d}')
    text_emb=mix_frame_emb(*e)
    frame,frame_latent=generate_baseon_prompt_emb(img_latent,text_emb,pbar)
    # img_latent=frame_latent
    frame.save(f"{hparam.output_dir}/{k:05d}.jpg")

  0%|          | 0/15000 [00:00<?, ?it/s]

### 方法2，使用前一帧的latent

In [ ]:
hparam.num_inference=50
hparam.start_step=10
hparam.fps=10
hparam.prompt_per_second=5
hparam.seed=17390125398225616219


os.makedirs(hparam.output_dir,exist_ok=True)


init_prompt,next_prompts=hparam.prompts[0],hparam.prompts[1:]
num_of_prompt=len(next_prompts)
num_of_generate_frame=(num_of_prompt-1)*hparam.prompt_per_second*hparam.fps
pbar=tqdm(range(num_of_generate_frame*hparam.num_inference+hparam.num_inference))

#初始化第一帧
_,frame_latent=generate_baseon_prompt_emb(img_latent,mix_frame_emb(0,1,1,0),pbar)

def mapfunc(t):
    '''
        把每一帧映射到对应的prompt索引,因为mix_frame_emb[0]不会被使用，所以需要 shift+1
        
        mix_frame_emb: [0,1,2,3,4,5,6]
          current_emb:   [0,1,2,3,4,5]
          
          current_emb[0]-->mix_frame_emb[1]
          current_emb[1]-->mix_frame_emb[2]
                  ....
          current_emb[5]-->mix_frame_emb[6]
        
    '''
    frac_index=t*(num_of_prompt-1)/(num_of_generate_frame-1)
    index1,index2=math.floor(frac_index),math.floor(frac_index)+1
    weight1,weight2=index2-frac_index,frac_index-index1
    
    if index2>num_of_prompt-1:
        index2=0
    return index1+1,index2+1,weight1,weight2  

hparam.start_step=0
for k,e in enumerate(map(mapfunc,range(num_of_generate_frame))):
    pbar.set_description(f'Frame {k:05d}')
    text_emb=mix_frame_emb(*e)
    frame,frame_latent=generate_baseon_prompt_emb(frame_latent,text_emb,pbar)
    # img_latent=frame_latent
    frame.save(f"{hparam.output_dir}/{k:05d}.jpg")

In [ ]:
# For video display:
from IPython.display import HTML

def create_movie(movie_name):
    if os.path.exists(movie_name): os.remove(movie_name)
    !ffmpeg -v 1 -y -f image2 -framerate {hparam.fps} -i {hparam.output_dir}/%05d.jpg -c:v libx264 -preset slow -qp 18 -pix_fmt yuv420p {movie_name}
# create_movie('grow_tree.mp4')

In [ ]:
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" %'imgs/grow_tree.mp4')